## Este é o segundo projeto proposto pelo curso de data science da TripleTen

# Título: Análise do risco de inadimplência dos mutuários

Este projeto foi realizado com o objetivo de testar conhecimentos em python, pandas, pré-processamento de dados em geral.

# Contents <a id='back'></a>

* [Introdução](#intro)
* [Visão geral dos dados](#data_review)
    * [Conclusões](#data_review_conclusions)
* [Pré-processamento de dados](#transformacao_dados)
* [Categorização de dados](#categorizando)
* [Testando as hipóteses](#hipoteses)
* [Conclusão geral](#conclusao)

## Introdução <a id='intro'></a>

Seu projeto é preparar um relatório para a divisão de empréstimos de um banco. Você precisará descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Seu relatório será considerado ao criar uma **pontuação de crédito** de um cliente em potencial. A **contagem de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.

## Visão geral dos dados <a id='data_review'></a>

Importando a biblioteca Pandas e salvando os dados no dataframe df.

In [258]:
# Carregando a biblioteca principal
import pandas as pd
from scipy import stats as st

In [259]:
# lendo os dados e salvando em df
df = pd.read_csv('credit_scoring_eng.csv')

### Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo

Antes de explorar nossos dados. Vamos exibir a quantidade de colunas e linhas

In [260]:
print(f'linhas: {df.shape[0]}, colunas: {df.shape[1]}')
# Quantidade de linhas e colunas do DataFrame

linhas: 21525, colunas: 12


Exibindo as primeiras 10 linhas do dataframe

In [261]:
# primeiras 10 linhas de df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Primeiras impressões: 

A coluna **days_employed** possue valores negativos. A coluna **education** tem alguns dados preenchidos em caixa alta. Essas duas colunas devem ser corrigidas.

In [262]:
# Obtendo informações sobre dados:
df.info()
# Na coluna total_income e days employed tem valores nulos
df.isna().sum()
# total de valores nulos em cada coluna:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

As colunas que possuem dados nulos são relacionadas a renda e dias trabalhados por isso precisamos avaliar se esses dados são nulos devido alguma informação correlacionada a outra coluna (exemplo: se as pessoas estão desempregadas)  ou simplesmente não foram preenchidos.

In [263]:
df[df['total_income'].isna()].value_counts('income_type')

income_type
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
dtype: int64

Filtrando o df para os valores que possuem `NaN` na coluna `total_income`, e depois disso olhando para os valores que aparecem em `income_type` (tipo de emprego), não vemos a opção desempregado. Então acredito que os dados não preenchidos estejam faltando por algum erro ao preencher os dados. Eles deveram ser preenchidos.

In [264]:
#Vamos olhar a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Os valores ausentes estão simétricos (ou seja, as mesmas linhas que faltam em days_employed faltam em total_income). Esses dados faltantes aconteceram para um número x de clientes, podemos verificar se  essa hipótese é verdadeira comparando se os dados faltantes nessas colunas são das mesmas linhas.

In [265]:
# Vamos aplicar algumas condições para filtrar dados e observar o número de linhas na tabela filtrada.
# true = 1 false = 0 se todas as comparações forem true a soma vai ser igual ao tamanho de df
if (df['days_employed'].isna() == df['total_income'].isna()).sum() == len(df):
    print('sim! todas as linhas dessas colunas são simétricas')
else:
    print('há diferenças entre essas linhas')
    
print(f"Número de linhas NaN em total income ={ df['total_income'].isna().sum() }")
print(f"Número de linhas NaN em days employed ={ df['days_employed'].isna().sum() }")
print('Porcentagem de valores nulos com relação ao total dos dados no df')
print((df['total_income'].isna().sum()*100/len(df)).round(3),'%')

sim! todas as linhas dessas colunas são simétricas
Número de linhas NaN em total income =2174
Número de linhas NaN em days employed =2174
Porcentagem de valores nulos com relação ao total dos dados no df
10.1 %


**Conclusão intermediária**

O número de linhas na tabela filtrada corresponde ao número de valores ausentes no dataframe.

No dataframe, 10% dos dados possuem valores ausentes. Devemos considerar se os dados ausentes podem ser devidos à característica específica do cliente. Vamos verificar se há alguma dependência de valores ausentes em relação ao valor de outros indicadores.

In [266]:
# clientes que não possuem dados sobre as características identificadas e a coluna com os valores ausentes
m_df = df[df['days_employed'].isna()]
# m_df é o dataframe que possuem apenas clientes com dados faltantes 

In [267]:
# Verificando a distribuição
# vamos olhar para cada coluna e verificar os diferentes valores e suas porcentagens em relação ao total de m_df.
for col in ['family_status', 'income_type', 'education', 'children','dob_years','gender','debt','purpose']:
    print(m_df[col].value_counts(dropna=False) / len(m_df))
    print('========')


married              0.568997
civil partnership    0.203312
unmarried            0.132475
divorced             0.051518
widow / widower      0.043698
Name: family_status, dtype: float64
employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64
secondary education    0.647654
bachelor's degree      0.228151
SECONDARY EDUCATION    0.030819
Secondary Education    0.029899
some college           0.025299
Bachelor's Degree      0.011500
BACHELOR'S DEGREE      0.010580
primary education      0.008740
Some College           0.003220
SOME COLLEGE           0.003220
Primary Education      0.000460
PRIMARY EDUCATION      0.000460
Name: education, dtype: float64
 0     0.661914
 1     0.218491
 2     0.093836
 3     0.016559
 20    0.004140
 4     0.003220
-1     0.001380
 5     0.000460
Name: children, dtype: float64
34    0.031739
40    0.030359
42    0.029899
31    0.029899
35    0.029439


Observações: 
Olhando para a coluna `family_status`  a maior porcentagem é casada.

coluna `income_type` a maioria são empregados.

coluna `education` a maior parte 65% tem ensino médio.

coluna `children` observo que a maioria não tem filhos (66%), e ainda tem dados problematicos como -1 e 20.

coluna que corresponde as idades `dob_years` esta bem distribuido.

Coluna sobre genero a maoria é feminina.

Coluna purpose que seria o motivo do emprestimo esta bem distribuido. Mas é necessário tratar os dados repetidos.

**Possíveis motivos para valores ausentes nos dados**

Não sei afirmar ainda se os dados faltantes são aleatórios ou se há algum motivo específicos agora podemos olhar para o df geral e ver se essas porcentagem continuam similares ou não.

In [268]:
# Verificar a distribuição em todo o conjunto de dados
for col in ['family_status', 'income_type', 'education', 'children','dob_years','gender','debt','purpose']:
    print(df[col].value_counts(dropna=False) / len(df))
    print('========')

married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64
employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64
secondary education    0.638792
bachelor's degree      0.219187
SECONDARY EDUCATION    0.035865
Secondary Education    0.033031
some college           0.031034
BACHELOR'S DEGREE      0.012729
Bachelor's Degree      0.012451
primary education      0.011614
Some College           0.002184
SOME COLLEGE           0.001347
PRIMARY EDUCATION      0.000790
Primary Education      0.000697
graduate degree        0.000186
Graduate Degree        0.000046
GRADUATE DEG

**Conclusão intermediária**

A distribuição no conjunto de dados original é semelhante à distribuição da tabela filtrada (com valores ausentes), acredito o erro nos dados é sistemico.

**Conclusões** <a id='data_review_conclusions'></a>

Olhando para a distribuição dos dados em cada coluna não encontrei nenhum padrão específico.

Os dados ausentes precisarão ser tratados considerando os valores preenchidos para cada categoria.

Próxima etapa será verificar as colunas e ver se precisam de correção nos dados preenchidos, retirar as duplicadas, corrigir os dados incorretos e depois tratar os valores ausentes.

## Pré-processamento de dados <a id='transformacao_dados'></a>

Vamos examinar cada coluna e corrigir oque for necessário.

Verificando os dados na coluna `education`:


In [269]:
# Vamos ver todos os valores na coluna de educação para verificar se e quais grafias precisarão ser corrigidas
df['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [270]:
# colocando os dados de forma padronizada
df['education'] = df['education'].str.lower()

In [271]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos
df['education'].value_counts()


secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

Verificando os dados na coluna `children`:

In [272]:
# Vamos ver a distribuição de valores na coluna `children`
print(df['children'].describe())
print('----------------------')
print(df['children'].value_counts())
print('----------------------')

data_problem = df[(df['children'] == 20) | (df['children'] == -1)]['children'].count()/(df['children'].count())
print(f'{data_problem*100}%')

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64
----------------------
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
----------------------
0.5714285714285714%


Existe um valor impossível que seria -1 para 47 pessoas e outro muito suspeito que no caso são 20 filhos para um número consideravel de pessoas (76 pessoas). Há algo provavelmente preenchido errado nestes dados. A porcentagem geral destes dados duvidosos é de 0.6%, de qualquer forma precisam ser tratados e corrigidos. 

O valor 20 provavelmente foi um erro de digitação até porque não existe valores intermediarios de 5 filhos até 20 filhos, acredito que valor 20 na verdade sejam de apenas 2 filhos. Neste caso vamos substituir os valores que estão 20 pelo valor 2.

Para o caso do valor -1 não podemos assumir que são apenas 1 filho, as vezes o número negativo pode indicar um dado faltante. Acreditamos que para este caso seria mais interessante substituir pela média ou mediana dos valores.


In [273]:
# primeiro vou substituir os dados que estavam com 20 por 2
df['children'] = df['children'].replace(20,2) #assumindo que foi erro de digitação
print(df['children'].value_counts()) # vamos olhar agora se foi corrigido
print(f" média {df['children'].mean()}") # agora vamos calcular a média 
print(f" mediana {df['children'].median()}")# agora vamos calcular a mediana
# -1 provavelmente é dado faltante, neste caso vou assumir a mediana 
df['children'] = df['children'].replace(-1,0)

 0    14149
 1     4818
 2     2131
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64
 média 0.4753542392566783
 mediana 0.0


In [274]:
# Verificando a coluna `children` novamente para ter certeza de que está tudo corrigido
df['children'].value_counts()


0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Verificando os dados da coluna `days_employed`:

In [275]:
# Analisando os dados problemáticos em `days_employed`
print(df['days_employed'].value_counts()) # valores negativos
print('--------------------------------------------')
print(df['days_employed'].describe()) 
# além disso os valores são enormes e provavelmente não estão em dias e sim em horas
# exemplo: o valor da média é de 63 mil dias, se dividirmos por 365 dias (1 ano) isso nos da aproximadamente 170 anos
# Assumindo que os dados estão em horas, se dividirmos os valores por 24h teremos valores mais consistentes com dias trabalhados
print('--------------------------------------------')
print(df['days_employed'].head(20)) 
print('--------------------------------------------')
print('A porcentagem dos valores negativos:')
print(df[df['days_employed'] < 0]['days_employed'].count()*100 / df['days_employed'].count()) # erro sistemico, dados devem ser corrigidos para numeros positivos
print('Porcentagem de valores maiores que 45 anos de trabalho:')
print(df[df['days_employed'] > 45 * 365]['days_employed'].count()*100 / df['days_employed'].count()) # valores maiores do que 45 anos trabalhando

-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64
--------------------------------------------
count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64
--------------------------------------------
0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
15      -972.364419
16     -1719.934226
17     -2369.999720
18    400281.136913
19    -10038.818549
Name: days_employed, 

A quantidade de dados problemática é de quase 100% e pode ser devido a alguns problemas técnicos. 
Provavelmente, os dias na verdade podem em horas, e o sinal de negativo entrou por algum erro técnico.

In [276]:
# Abordando os valores problemáticos
df['days_employed'] = df['days_employed'].abs() # colocandos os dados em valores positivos
df['days_employed'] = df['days_employed']/24 # acredito que os valores estavam em horas por isso dividi por 24 para ficar em dias trabalhados

In [277]:
# Verificando o resultado - vamos verificar se esta corrigido
print(df['days_employed'].describe())

print(df[df['days_employed'] < 0]['days_employed'].count()*100 / df['days_employed'].count()) 
print(df[df['days_employed'] > 45 * 365]['days_employed'].count()*100 / df['days_employed'].count())
df[df['days_employed'] == df['days_employed'].max()] # curiosidade, a pessoa com o valor maior em dias trabalhados

count    19351.000000
mean      2788.113704
std       5792.953355
min          1.005901
25%         38.625386
50%         91.425857
75%        230.745102
max      16739.808353
Name: days_employed, dtype: float64
0.0
1.9120458891013383


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,16739.808353,56,secondary education,1,widow / widower,2,F,retiree,0,28204.551,housing renovation


Vamos agora olhar para a coluna  `dob_years`

In [278]:
# Verificando o `dob_years` para valores suspeitos e a porcentagem
print(df['dob_years'].value_counts())
print(df['dob_years'].describe()) # existem alguns valores com 0
print(df[df['dob_years'] == 0]['dob_years'].count()*100/len(df['dob_years'])) # 0.46% dos dados estao com valores 0 anos
media_age = df[df['dob_years'] > 0 ]['dob_years'].mean() #calculando a média das idades
print(media_age)

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64
0.4692218350754936
43.497479462285284


Substituiremos os valores 0 pela média das idades

In [279]:
# Resolvendo o problema na coluna `dob_years`
df['dob_years'] = df['dob_years'].replace(0,media_age)

In [280]:
# Verificando o resultado
print(df['dob_years'].value_counts())
print(df['dob_years'].describe())

35.000000    617
40.000000    609
41.000000    607
34.000000    603
38.000000    598
42.000000    597
33.000000    581
39.000000    573
31.000000    560
36.000000    555
44.000000    547
29.000000    545
30.000000    540
48.000000    538
37.000000    537
50.000000    514
43.000000    513
32.000000    510
49.000000    508
28.000000    503
45.000000    497
27.000000    493
56.000000    487
52.000000    484
47.000000    480
54.000000    479
46.000000    475
58.000000    461
57.000000    460
53.000000    459
51.000000    448
59.000000    444
55.000000    443
26.000000    408
60.000000    377
25.000000    357
61.000000    355
62.000000    352
63.000000    269
64.000000    265
24.000000    264
23.000000    254
65.000000    194
22.000000    183
66.000000    183
67.000000    167
21.000000    111
43.497479    101
68.000000     99
69.000000     85
70.000000     65
71.000000     58
20.000000     51
72.000000     33
19.000000     14
73.000000      8
74.000000      6
75.000000      1
Name: dob_year

Agora vamos verificar a coluna `family_status`

In [281]:
# Vamos ver os valores da coluna
df['family_status'].value_counts() # aparentemente estão todos consistentes

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

Agora vamos verificar a coluna `gender`

In [282]:
# Vamos ver os valores na coluna
df['gender'].value_counts() # aparentemente estão todos consistentes

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Agora vamos verificar a coluna `income_type`

In [283]:
# Vamos ver os valores na coluna
df['income_type'].value_counts() # aparentemente estão consistentes 

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

Agora vamos ver se temos duplicatas em nossos dados

In [284]:
# Verificando duplicatas
print(df.duplicated().sum()) # temos  71 dados duplicados nas linhas de df
df[df.duplicated()]

71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41.0,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,NaN,58.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,NaN,34.0,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,NaN,60.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,NaN,58.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64.0,secondary education,1,married,0,F,retiree,0,NaN,supplementary education
21032,0,NaN,60.0,secondary education,1,married,0,F,retiree,0,NaN,to become educated
21132,0,NaN,47.0,secondary education,1,married,0,F,employee,0,NaN,housing renovation
21281,1,NaN,30.0,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate


In [285]:
# retirando os dados duplicados e resetando os index 
df = df.drop_duplicates().reset_index(drop=True)

In [286]:
# Última verificação se temos duplicatas
df.duplicated().sum()

0

In [287]:
#Verificando o tamanho do conjunto de dados após primeiras manipulações com ele
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


Agora os dados preenchidos nas colunas estão corrigidos e retiramos as linhas duplicadas, podemos agora tratar os valores ausentes das colunas days_employed e total_income para obter o df final para a analise.


### Trabalhando com valores ausentes

#### Restaurar valores ausentes em `total_income`

Precisamos corrigir os valores em **total_income** e **days_employed.**

In [288]:
# Vamos escrever uma função que calcule a categoria de idade
# para esta função vamos separar em 3 grupos: jovens, adultos e idosos
df['dob_years'].value_counts()

def age_group(age):
    if age <= 19:
        return 'young'
    if age <= 59:
        return 'adult'
    return 'elderly' 

In [289]:
# Testando se a função funciona
age_group(18)

'young'

In [290]:
# vamos criar uma coluna nova com base na função

df['age_group'] = df['dob_years'].apply(age_group)

In [291]:
# Verificação dos valores na nova coluna
df['age_group'].value_counts()


adult      18940
elderly     2500
young         14
Name: age_group, dtype: int64

Vamos considerar neste caso que as fases da vida de cada pessoa interfere tanto no tempo de trabalho (days_employed) quanto na renda (total_income). Considerando que jovens quando conseguem seu primeiro emprego não tem um salário tão alto quanto de adultos formados e com experiencia de trabalho, e idosos geralmente já estão aposentados. Então vamos considerar as idades desses grupos para preencher os valores ausentes nestas colunas.

Vamos criar uma tabela que tenha apenas dados sem valores ausentes. Esses dados serão usados para restaurar os valores ausentes

In [292]:
# tabela sem valores ausentes (df_notnull)
df_notnull = df.dropna()
print(df_notnull.shape) # tamanho da tabela (linhas,coluna)
df_notnull.head() 

(19351, 13)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,351.569709,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,167.700156,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,234.309275,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,171.864467,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,14177.753002,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult


In [293]:
# Vejamos os valores médios de renda com base em seus fatores identificados
mean = df_notnull.groupby('age_group')['total_income'].mean() # agrupando por idade, a média para cada uma em total_income
print(mean) 
print('---------------------')
print(df_notnull[df_notnull['age_group'] == 'young']['total_income'].describe()) #só de curiosidade vamos olhar separadamente jovens
print('---------------------')
print(df_notnull[df_notnull['age_group'] == 'adult']['total_income'].describe()) #só de curiosidade vamos olhar separadamente adultos
print('---------------------')
print(df_notnull[df_notnull['age_group'] == 'elderly']['total_income'].describe()) #só de curiosidade vamos olhar separadamente idosos

age_group
adult      27292.133454
elderly    23021.639994
young      16993.942462
Name: total_income, dtype: float64
---------------------
count       13.000000
mean     16993.942462
std       5567.118012
min       9459.851000
25%      12929.944000
50%      14934.901000
75%      21009.404000
max      26753.823000
Name: total_income, dtype: float64
---------------------
count     17083.000000
mean      27292.133454
std       16608.577732
min        3306.762000
25%       16884.713000
50%       23655.940000
75%       33147.920000
max      362496.645000
Name: total_income, dtype: float64
---------------------
count      2255.000000
mean      23021.639994
std       14930.221646
min        3471.216000
25%       13725.605500
50%       19761.425000
75%       27953.500500
max      274402.943000
Name: total_income, dtype: float64


In [294]:
# Vejamos os valores medianos de renda com base em seus fatores identificados
median = df_notnull.groupby('age_group')['total_income'].median() # agrupando novamente pelas idades,a mediana de cada grupo de idade
print(median)

age_group
adult      23655.940
elderly    19761.425
young      14934.901
Name: total_income, dtype: float64


As médias tem valores mais altas que as medianas. Mediana é menos sensível a outliers

Neste caso vamos utilizar as medianas pois eles presentam melhor os valores da maioria dos clientes. Já que existem valores máximos muito acima da média acreditamos que a mediana deixara os dados mais próximo da realidade de renda destes clientes.

In [295]:
# Escrevendo uma função que usaremos para preencher os valores ausentes
def fill_na_median(row,median): 
    if not pd.notnull(row['total_income']):
        if row['age_group'] == 'young':
            row['total_income'] = median['young']
            return row
        if row['age_group'] == 'adult':
            row['total_income'] = median['adult']
            return row
        row['total_income'] = median['elderly']
    return row

In [296]:
fill_na_median(df.iloc[26],median)  #escolhi uma linha que tem valor ausente na coluna total_income para testar

<ipython-input-295-cdd6b6ffc3fe>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['total_income'] = median['adult']


children                              0
days_employed                       NaN
dob_years                          41.0
education           secondary education
education_id                          1
family_status                   married
family_status_id                      0
gender                                M
income_type               civil servant
debt                                  0
total_income                   23655.94
purpose                       education
age_group                         adult
Name: 26, dtype: object

In [297]:
fill_na_median(df.loc[12],median)  # testando outra linha com dado ausente agora para elderly

<ipython-input-295-cdd6b6ffc3fe>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['total_income'] = median['elderly']


children                              0
days_employed                       NaN
dob_years                          65.0
education           secondary education
education_id                          1
family_status         civil partnership
family_status_id                      1
gender                                M
income_type                     retiree
debt                                  0
total_income                  19761.425
purpose               to have a wedding
age_group                       elderly
Name: 12, dtype: object

In [298]:
df['total_income']= df['total_income'].fillna(df.groupby('age_group')['total_income'].transform('median'))
# https://stackoverflow.com/questions/19966018/pandas-filling-missing-values-by-mean-in-each-group
df['total_income'].value_counts() #checando valores

23655.940    1858
19761.425     246
31791.384       2
42413.096       2
14934.901       2
             ... 
23686.835       1
9606.294        1
28156.762       1
24931.195       1
13127.587       1
Name: total_income, Length: 19348, dtype: int64

In [299]:
# Verificando se temos algum erro
print(df.iloc[26]) # essa linha foi preenchida corretamente
df['total_income'].isna().sum() # somando todos os valores NaN se ainda existirem, como o valor é 0 significa que todos foram preenchidos

children                              0
days_employed                       NaN
dob_years                          41.0
education           secondary education
education_id                          1
family_status                   married
family_status_id                      0
gender                                M
income_type               civil servant
debt                                  0
total_income                   23655.94
purpose                       education
age_group                         adult
Name: 26, dtype: object


0

Agora que terminamos com `total_income`, vamos verificar se o número total de valores nesta coluna corresponde ao número de valores em outras.

In [300]:
# Verificar o número de entradas nas colunas
df.count()

children            21454
days_employed       19351
dob_years           21454
education           21454
education_id        21454
family_status       21454
family_status_id    21454
gender              21454
income_type         21454
debt                21454
total_income        21454
purpose             21454
age_group           21454
dtype: int64

####  Restaurar valores em `days_employed`

Vamos seguir a mesma linha de raciocínio utilizado na coluna anterior

In [301]:
# Distribuição de `days_employed` medianos com base em seus parâmetros identificados
median_days = df_notnull.groupby('age_group')['days_employed'].median()
print(median_days)
print(df_notnull[df_notnull['age_group'] == 'young']['days_employed'].describe()) #só de curiosidade vamos olhar separadamente jovens
print('---------------------')
print(df_notnull[df_notnull['age_group'] == 'adult']['days_employed'].describe()) #só de curiosidade vamos olhar separadamente adultos
print('---------------------')
print(df_notnull[df_notnull['age_group'] == 'elderly']['days_employed'].describe()) #só de curiosidade vamos olhar separadamente idosos

age_group
adult         77.546588
elderly    14801.234092
young         30.187192
Name: days_employed, dtype: float64
count    13.000000
mean     26.403254
std      12.326602
min       4.656783
25%      21.248747
50%      30.187192
75%      33.207651
max      42.507630
Name: days_employed, dtype: float64
---------------------
count    17083.000000
mean      1582.229989
std       4507.950762
min          1.005901
25%         34.830871
50%         77.546588
75%        164.624878
max      16739.808353
Name: days_employed, dtype: float64
---------------------
count     2255.000000
mean     11939.339310
std       6263.726617
min          4.179559
25%      13851.418287
50%      14801.234092
75%      15742.537523
max      16738.158823
Name: days_employed, dtype: float64


In [302]:
# Distribuição de `days_employed` médios com base em seus parâmetros identificados
mean_days = df_notnull.groupby('age_group')['days_employed'].mean()
print(mean_days)

age_group
adult       1582.229989
elderly    11939.339310
young         26.403254
Name: days_employed, dtype: float64


Neste caso, usaremos os valores da média pois eles fazem mais sentido para o grupo de adultos

In [303]:
# Vamos escrever uma função que calcule médias com base no seu parâmetro identificado
def fill_na_mean(row,mean):
    if not pd.notnull(row['days_employed']):
        if row['age_group'] == 'young':
            row['days_employed'] = mean['young']
            return row
        if row['age_group'] == 'adult':
            row['days_employed'] = mean['adult']
            return row
        row['days_employed'] = mean['elderly']
    return row

In [304]:
# Verifique se a função funciona 
fill_na_mean(df.loc[26],mean_days) #testando em uma linha com valor nulo

<ipython-input-303-ce2ec7bb7797>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['days_employed'] = mean['adult']


children                              0
days_employed               1582.229989
dob_years                          41.0
education           secondary education
education_id                          1
family_status                   married
family_status_id                      0
gender                                M
income_type               civil servant
debt                                  0
total_income                   23655.94
purpose                       education
age_group                         adult
Name: 26, dtype: object

In [305]:
# Aplicando a função ao days_employed --- aplicando para todas as linhas
df['days_employed'] = df['days_employed'].fillna(df.groupby('age_group')['days_employed'].transform('mean'))

In [306]:
print(df.iloc[12])
df['days_employed'].isna().sum()  # foi preenchido para todas as linhas

children                              0
days_employed               11939.33931
dob_years                          65.0
education           secondary education
education_id                          1
family_status         civil partnership
family_status_id                      1
gender                                M
income_type                     retiree
debt                                  0
total_income                  19761.425
purpose               to have a wedding
age_group                       elderly
Name: 12, dtype: object


0

Verificando se o número total de valores nesta coluna corresponde ao número de valores em outras.

In [307]:
# Verificando as entradas em todas as colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


## Categorização de dados <a id=categorizando><a/>

Agora que os dados estão prontos para a análise vamos visualizar os valores das colunas principais, como precisaremos descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre a inadimplência de um empréstimo. Valos olhar para as colunas do estado civil `family_status`, número de filhos `children` e se o cliente ja deveu emprestimos `debt`.

In [308]:
# Exiba os valores dos dados selecionados para categorização
print(df['children'].value_counts()) #valores para quantidade de filhos
print('-----------------------------')
print(df['family_status'].value_counts())# estado civil
print('-----------------------------')
print(df['family_status_id'].value_counts())# id estado civil
print('-----------------------------')
print(df['debt'].value_counts()) # devedores 
print('-----------------------------')

0    14138
1     4808
2     2128
3      330
4       41
5        9
Name: children, dtype: int64
-----------------------------
married              12339
civil partnership     4151
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64
-----------------------------
0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64
-----------------------------
0    19713
1     1741
Name: debt, dtype: int64
-----------------------------


No caso de Children temos de 0 a 5, estado civil esta dividido em 5 tipos e debitos esta com 0 (0 nao deve) ou 1 (deve)

Pensando nos tipos de status familiar podemos juntar os casados e morando juntos em uma mesma categoria que seria em um relacionamento (logo provavelmente eles dividem as contas) e os outros permanecem com os mesmos status

In [309]:
# Vamos escrever uma função para categorizar os dados com base em tópicos comuns
def relationship(v):
    return 'with_relationship' if v == 'married' or v == 'civil partnership' else v


In [310]:
# Criando uma coluna com a categoria e contando os valores para elas
df['relationship'] = df['family_status'].apply(relationship)
print(df['relationship'].value_counts()) # quantidade de pessoas para cada valor

relationship_statistics = df['relationship'].value_counts()*100/df['relationship'].count()
print(relationship_statistics) # 76.8% estão em um relacionamento


with_relationship    16490
unmarried             2810
divorced              1195
widow / widower        959
Name: relationship, dtype: int64
with_relationship    76.862124
unmarried            13.097791
divorced              5.570057
widow / widower       4.470029
Name: relationship, dtype: float64


Agora vamos olhar para a coluna children

In [311]:
# Examinando os dados
children = pd.DataFrame(df['children'].describe())
print(children)

           children
count  21454.000000
mean       0.478372
std        0.756009
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        5.000000


In [312]:
# Obtendo estatísticas resumidas para a coluna
children_statistics = df['children'].value_counts()*100/df['children'].count()
print(children_statistics) # 65,8%  não tem filhos

0    65.899133
1    22.410739
2     9.918896
3     1.538175
4     0.191107
5     0.041950
Name: children, dtype: float64


Podemos criar uma coluna nova com base na coluna children, onde podemos dizer se o cliente tem filhos ou não tem filhos, no caso se for 0 NÃO TEM se for 1 TEM.

In [313]:
# Criando uma função para categorização da coluna tem filhos (has children)
def children(c):
    return 1 if c>0 else 0

In [314]:
# Criando a coluna com a categoria
df['has_children'] = df['children'].apply(children)

In [315]:
# Contando os valores de cada categoria para ver a distribuição
df['has_children'].value_counts()

0    14138
1     7316
Name: has_children, dtype: int64

In [316]:
# verificando o df
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,relationship,has_children
0,1,351.569709,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult,with_relationship,1
1,1,167.700156,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,with_relationship,1
2,0,234.309275,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,with_relationship,0
3,3,171.864467,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,with_relationship,1
4,0,14177.753002,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult,with_relationship,0


Agora que o df está pronto e temos novas colunas com categorias podemos verificar as hipóteses

## Verificando as Hipóteses <a id='hipoteses'><a/>


**Existe uma correlação entre o número de filhos e do pagamento em dia?**

In [317]:
# Verificando os dados se tem crianças e do pagamento em dia
print(df.groupby('has_children')['debt'].value_counts(normalize=True))

has_children  debt
0             0       0.924742
              1       0.075258
1             0       0.907463
              1       0.092537
Name: debt, dtype: float64


Aqui esta mostrando que clientes sem filhos tem 2 pontos percentuais a mais de chances de pagar em dia do que clientes com filhos.

In [318]:
print(df['children'].value_counts())
# Calculando a taxa de inadimplência com base no número de filhos
print(df.groupby('children')['debt'].value_counts(normalize=True))

0    14138
1     4808
2     2128
3      330
4       41
5        9
Name: children, dtype: int64
children  debt
0         0       0.924742
          1       0.075258
1         0       0.907654
          1       0.092346
2         0       0.905075
          1       0.094925
3         0       0.918182
          1       0.081818
4         0       0.902439
          1       0.097561
5         0       1.000000
Name: debt, dtype: float64


Vamos fazer um teste estatístico para descobrir se a diferença percentual é significativa ou não na análise entre ter filhos ou não e pagar em dia.

In [319]:
has_children = df[df['has_children'] == 1]['debt'].values
no_children = df[df['has_children'] == 0]['debt'].values

**Hipóteses:**

**1) Hipótese nula: Não existe valor estatístico relevante para considerar uma diferença na média de pessoas que tem filhos e tem dívida das pessoas que tem filhos e não tem dívida.**

caso esta hipótese seja verdadeira: 'Nós não podemos rejeitar a hipótese nula'

**2) Hipótese alternativa: Existe diferença estatística entre pessoas que tem filhos e não tem dívida.**

caso esta hipótese seja verdadeira: 'Rejeitamos a hipótese nula'


In [320]:
results = st.ttest_ind(
has_children,
no_children)

print('p-value: ', results.pvalue)

alpha = 0.05

if results.pvalue < alpha:
    print('Rejeitamos a hipótese nula')
else:
    print("Nós não podemos rejeitar a hipótese nula")

p-value:  1.1110998973945563e-05
Rejeitamos a hipótese nula


**Conclusão intermediária**

A quantidade de filhos parece influenciar nas estatísticas, por isso, fizemos um ttest e foi encontrado uma diferença entre devedores que tem filhos e devedores que não tem filhos. Por tanto, indica que a pontuação de crédito para uma pessoa que tem filhos e deve ser diferente.

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [321]:
# Verificando os dados de status da família e do pagamento em dia
print(df.groupby('relationship')['debt'].value_counts(normalize=True))

# Calculando a taxa padrão com base no status da família
print(df.groupby('family_status')['debt'].value_counts(normalize=True))


relationship       debt
divorced           0       0.928870
                   1       0.071130
unmarried          0       0.902491
                   1       0.097509
widow / widower    0       0.934307
                   1       0.065693
with_relationship  0       0.920012
                   1       0.079988
Name: debt, dtype: float64
family_status      debt
civil partnership  0       0.906529
                   1       0.093471
divorced           0       0.928870
                   1       0.071130
married            0       0.924548
                   1       0.075452
unmarried          0       0.902491
                   1       0.097509
widow / widower    0       0.934307
                   1       0.065693
Name: debt, dtype: float64


**Conclusão intermediária**

Para os diferentes status familiar, podemos perceber que os melhores pagadores são os viuvos (widow / widower). E os maiores devedores são unmarried e civil partnerhip, que tem taxas maiores (isso pode estar associado com pessoas mais jovens, por exemplo)

**Existe uma correlação entre a renda do cliente e o pagamento em dia?**

In [322]:
# Verificando os dados do nível de renda
print(df['total_income'].describe())
# vamos fazer uma função dividindo os clientes em 4 categorias, Q1 os de renda no valor de até 25%
# Q2 de até 50%, Q3 de até 75% e Q4% acima de 75% até o valor máximo, com isso podemos dividir os clientes pelo nivel de renda
def income(value):
    income_cat = pd.DataFrame(df['total_income'].describe()).T
    if value < income_cat['25%'].values:
        return 'Q1'
    elif value < income_cat['50%'].values:
        return 'Q2'
    elif value < income_cat['75%'].values:
        return 'Q3'
    else:
        return 'Q4'
# criando uma nova coluna com a categoria de renda dividida em Q1,Q2,Q3 e Q4.
df['income_Q'] = df['total_income'].apply(income)
df['income_Q'].value_counts()

count     21454.000000
mean      26435.713612
std       15688.437744
min        3306.762000
25%       17213.621250
50%       23655.940000
75%       31330.237250
max      362496.645000
Name: total_income, dtype: float64


Q3    5878
Q4    5364
Q1    5364
Q2    4848
Name: income_Q, dtype: int64

In [323]:
# Calcular a taxa de inadimplência com base no nível de renda
print(df.groupby('income_Q')['debt'].value_counts(normalize=True))

income_Q  debt
Q1        0       0.920395
          1       0.079605
Q2        0       0.914191
          1       0.085809
Q3        0       0.912385
          1       0.087615
Q4        0       0.928598
          1       0.071402
Name: debt, dtype: float64


**Conclusão intermediária**

Para as diferentes rendas dos clientes, as taxas não diferem significativamente.

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [324]:
# Conferindo os percentuais de inadimplência para cada finalidade de crédito
# print(df['purpose'].str.contains('wedding').sum()) #descobrindo quantas linhas tem a palavra chave wedding
print(df.groupby('purpose')['debt'].value_counts(normalize=True))

def cat(x):
    if 'wedding'in x:
        return 'wedding'
    if 'real estate' in x or 'housing' in x or 'house' in x or 'property' in x:
        return 'real estate transaction'
    if 'car' in x:
        return 'car'
    if 'education' in x or 'university' in x or 'educated' in x:
        return 'education'
    
df['motive category'] = df['purpose'].apply(cat)
print(df.groupby('motive category')['debt'].value_counts(normalize=True))

purpose                           debt
building a property               0       0.912763
                                  1       0.087237
building a real estate            0       0.923077
                                  1       0.076923
buy commercial real estate        0       0.928896
                                            ...   
transactions with my real estate  1       0.079745
university education              0       0.911504
                                  1       0.088496
wedding ceremony                  0       0.919090
                                  1       0.080910
Name: debt, Length: 76, dtype: float64
motive category          debt
car                      0       0.906410
                         1       0.093590
education                0       0.907800
                         1       0.092200
real estate transaction  0       0.927666
                         1       0.072334
wedding                  0       0.919966
                         1       0.08

**Conclusão intermediária**

Os percentuais de inadimplência para os diferentes motivos dos emprestimos estão com uma diferença de até 3 pontos percentuais, no caso os motivos que ocorrem maior inadimplencia seriam a compra de carro e educação que pode estar relacionado a pessoas mais jovens.

# Conclusão Geral <a id='conclusao'></a>

   O objetivo deste relatório era analisar um dataframe de dados dos clientes enviado pelo banco, para testar as seguintes hipóteses: se o número de filhos e se o status familiar tem algum impacto na inadimplência do pagamento do empréstimo. Após o tratamento dos dados e categorização dos mesmos pudemos testar essas hipóteses. 
    Conseguimos encontrar valores estatísticos diferentes na análise para clientes que tem filhos e também para clientes de diferentes status familiar.
    Concluimos que a quantidade de filhos parece influenciar nas estatísticas,indicando que a pontuação de crédito para uma pessoa que tem filhos e deve ser diferente, já que ela tende a ser mais inadimplente. Para os diferentes status familiar, os melhores pagadores são os viuvos e os maiores devedores são pessoas que não tem um relacionamento (que pode ser associados a pessoas mais jovens). 
    
    
* Caso tenham interesse:
    
   Podemos re-avaliar o tratamento de dados conforme sugestões, verificando algumas formas de preencher os dados ausentes e os dados com erro técnico para assim analisar novamente e testar essas hipóteses.
   Também podemos testar novas hipóteses como tipo de profissão e educação (grau de escolaridade) dos clientes, além de verificar hipóteses para diferentes grupos de idades e se terá algum impacto na inadimplência.